# Preprocess module

## Import Jupyter Module in other files

this code is for importing .ipynb files. YEYEEE

- code form <br/ > 
http://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Importing%20Notebooks.html

In [1]:
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

In [2]:
def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

In [3]:
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod

In [4]:
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

In [5]:
sys.meta_path.append(NotebookFinder())

- word2vec 모듈 다운로드가 요구됨

In [6]:
import format_module
import bookingreview
import preprocessreview
import wordvectormaker
import postag_module
import gensim
import xl_to_br_module

importing Jupyter notebook from format_module.ipynb


C:\Program Files\Anaconda\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


# 데이터 저장

In [7]:
import pickle

def save_object(obj, filename):
    with open(filename, 'wb') as output:
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

def load_object(filename):
    with open(filename, 'rb') as input:
        return pickle.load(input)

# Main

In [8]:
def postagModule(xl_name, reviewDB):
    BRList = xl_to_br_module.xl_to_BookingReview(xl_name)
    reviewDB.add_review_list(BRList)
    
    PRList_twitter = postag_module.twitter(BRList)
    PRList_komoran = postag_module.komoran(BRList)
    save_object(PRList_twitter, "save_PR_twitter.pkl")
    save_object(PRList_komoran, "save_PR_komoran.pkl")
    save_object(reviewDB, "save_reviewDB.pkl")
    
    return (PRList_twitter, PRList_komoran)

In [9]:
def embeddingModule(PRList):
    word2vec_model = gensim.models.Word2Vec.load('./models/namuwiki_testmodel_Komoran.model')
    return wordvectormaker.word2vec_to_PreprocessReview_list(word2vec_model, PRList)

In [10]:
def formattingModule(PRList, reviewDB):
    formatted_list = [FormattedReview(review) for review in PRList]
    save_object(formatted_list, "save_formatted_review.pkl")
    return formatted_list

In [11]:
def preprocessModule(xl_name, reviewDB):
    (PRList_twitter, PRList_komoran) = postagModule(xl_name, reviewDB)
    embeddingModule(PRList_komoran)
    formatted_list = formattingModule(PRList_komoran, reviewDB)
    print("result : " + formatted_list.__str__())

In [21]:
reviewDB = format_module.ReviewDB()
preprocessModule("Commonreviews_snuproject _test.xlsx", reviewDB)

FileNotFoundError: [Errno 2] No such file or directory: './models/namuwiki_testmodel'

In [12]:
import bookingreview
import format_module

reviewDB = format_module.ReviewDB()
reviewList = [bookingreview.BookingReview("회사", "id", 10, "asdfsadf", 40000.236+i*0.001, True, i) for i in range(50000)]

reviewDB.add_review_list(reviewList)
reviewDB.size()

KeyboardInterrupt: 